# Time Series Forcasting w/ Python

## Content
- Time series analysis
    - ACF plot
    - PACF plot
    - Time seriees decomposition
- Statistical tests
    - ADF test - Test for stationarity
    - Ljung-Box test - Residuals analysis
    - Granger causality - Multivariate forecasting
- Forcasting - Statistical models
    - Moving average model - MA(q)
    - Autoregressive model - AR(q)
    - ARMA(p,q)
    - ARIMA(p,d,q)
    - SARIMA(p,d,q)(P,D,Q)$_m$
    - SARIMAX
    - VARMAX
    - BATS and TBATS
    - Exponential smoothing
- Forcasting - Deep learning models
    - Deep neural network (DNN)
    - Long short-term memory - LSTM
    - Convolutional neural network - CNN
    - Autoregressive deep learning model

### Time series analysis

The autocorrelation function (ACF) plot shows the autocorrelation coefficients as a function of the lag.

Example use case:
- Use it to determine the order q of a stationary MA(q) process
- A stationary MA(q) process has significant coefficients up until lag q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# Parameters
n = 500     # Number of observations
q = 3       # MA order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define MA coefficients (e.g., θ1=2.0, θ2=3.0, θ3=4.0)
thetas = np.arange(2.0, 2.0 + q)

# Generate MA(q) time series
ma_series = np.array([
    sum(white_noise[i - j] * thetas[j] for j in range(q) if i - j >= 0)
    for i in range(n)
])

# Plot the autocorrelation function (ACF)
plt.figure(figsize=(10, 4))
plot_acf(ma_series, lags=40)
plt.title(f"Autocorrelation Function of MA({q}) Process")
plt.tight_layout()
plt.show()
